In [665]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
} 

<IPython.core.display.Javascript object>

In [666]:
#File:CountySnapshots.ipynb
#Author: Rafer Cooley
#Desc:Generate County Snapshots of Juvenile Data
from ipywidgets import widgets
from ipywidgets import interactive
from IPython.display import display, HTML, clear_output
#from pivottablejs import pivot_ui
import matplotlib.pyplot as plt
import csv
import numpy as np
import pandas as pd
import os, sys
import subprocess
import base64 #for createdownloadlink
#import data_functions as dfunct

import plotly as pl
import plotly.figure_factory as ff
import plotly.graph_objs as go

pl.offline.init_notebook_mode(connected=True)

%run getdata.ipynb
#%run crimestats.ipynb

import warnings
warnings.filterwarnings('ignore')

             Population 11-17  Total    Rate
County Year                                 
Albany 2010              2112   96.0  454.55
Year
2012         NaN
2013    0.597430
2014   -0.068330
2015   -0.056888
2016   -0.078969
Name: pct, dtype: float64
[713.51, 672.92, 619.78, 633.37]


In [667]:
my_styles = """
<style>
@media screen{
.container { width:90% !important; }
.page-break {page-break-after:always;}
}
@media print{
.container { width:90% !important; }
.page-break {page-break-after:always;}
}
</style>

"""
display(HTML(my_styles))

In [668]:
dfunct = DataFunctions()
overview = dfunct.getOverview()#not used
dfs = dfunct.getDFS()#afcars, county, plc

agegroup_demographic_data = dfunct.getDemographic_By_AgeGroup()
population_data = dfunct.getPopulationData()#phasing out?
#arrest_totals = dfunct.getStateTotalArrests(juvenile_arrests,population_data)

school = dfunct.getSchool()
school_county_rates = dfunct.getSchool_county_rates(school,agegroup_demographic_data)
school_state_rates = dfunct.getSchool_state_rates(school,agegroup_demographic_data)

juvenile_arrests = dfunct.getORIData(agegroup_demographic_data)
arrest_totals = dfunct.getStateTotalArrests(juvenile_arrests,agegroup_demographic_data)

judicial_district_population_data = dfunct.getJudicialDistrictJuvenilePopulations(agegroup_demographic_data)
placement_data = dfunct.getJudicialPlacementData()
placement_rates_data,state_placement_rates_data = dfunct.getJudicialPlacementRates(placement_data,judicial_district_population_data)

demographic_data = dfunct.getDemographicData()
court_case_counts = dfunct.getCourtCaseNumbersData()

data_sources_for_citation = [
    
]

#qualifiers for ORI data
violent_felony = ['Manslaughter','Rape','Robbery','Aggravated-Assault']
lesser_offenses = ['Burglary','Larceny-Theft','Motor-Vehicle-Theft','Arson',
    'Other-Assaults','Forgery-Counterfeiting','Fraud','Embezzlement','Stolen-Property','Vandalism','Weapons',
    'Prostitution','Sex-Offenses','Drug-Abuse-Violations','Drug-Abuse-Violations-Possession','Gambling-Offenses',
    'Offenses-Against-Family-Children','Driving-Under-the-Influence','Liquor-Laws','Drunkenness',
    'Disorderly-Conduct','All-Other-Offenses-Except-Traffic','Suspicion','Curfew-Loitering-Law-Violations',
    'Run-a-ways','Vagrancy']

#qualifiers for placements data
placement_county_districts = []
placement_county_districts.append(['Laramie'])
placement_county_districts.append(['Albany','Carbon'])
placement_county_districts.append(['Uinta','Sweetwater','Lincoln'])
placement_county_districts.append(['Johnson','Sheridan'])
placement_county_districts.append(['Big Horn','Park','Hot Springs','Washakie'])
placement_county_districts.append(['Campbell','Weston','Crook'])
placement_county_districts.append(['Natrona'])
placement_county_districts.append(['Converse','Niobrara','Goshen','Platte'])
placement_county_districts.append(['Teton','Fremont','Sublette'])

In [669]:
#Plotly PDF Reporting Helpers
#https://plot.ly/python/pdf-reports/
def report_block_template(report_type, graph_url, caption=''):
    if report_type == 'interactive':
        graph_block = '<iframe style="border: none;" src="{graph_url}.embed" width="100%" height="600px"></iframe>'
    elif report_type == 'static':
        graph_block = (''
            '<a href="{graph_url}" target="_blank">' # Open the interactive graph when you click on the image
                '<img style="height: 400px;" src="{graph_url}.png">'
            '</a>')

    report_block = ('' +
        graph_block + 
        '{caption}' + # Optional caption to include below the graph
        '<br>'      + # Line break
        '<a href="{graph_url}" style="color: rgb(190,190,190); text-decoration: none; font-weight: 200;" target="_blank">'+ 
            'Click to comment and see the interactive graph' + # Direct readers to Plotly for commenting, interactive graph
        '</a>' + 
        '<br>' + 
        '<hr>') # horizontal line                       

    return report_block.format(graph_url=graph_url, caption=caption)

In [670]:
#Show html table functions
idx = pd.IndexSlice

def display_afcars_html(county,year_tup):
    sources = []
    try:
        display(HTML('<h3>Afcars Data</h3>'))
        display(HTML(dfs[0].loc[idx[county,year_tup[0]:year_tup[1]], :].transpose().to_html()))
    except Exception as e:
        print('Error showing DFS data.'+str(e))
        
def display_county_html(county,year_tup):
    try:
        display(HTML('<h3>DFS By County</h3>'))
        display(HTML(dfs[1].loc[idx[county,year_tup[0]:year_tup[1]], :].transpose().to_html()))
    except Exception as e:
        print('Error showing DFS data.'+str(e))
        
#Table Columns
#Manslaughter,Rape,Robbery,Aggravated-Assault,Burglary,Larceny-Theft,Motor-Vehicle-Theft,Arson
#Other-Assaults,Forgery-Counterfeiting,Fraud,Embezzlement,Stolen-Property,Vandalism,Weapons
#Prostitution,Sex-Offenses,Drug-Abuse-Violations,Drug-Abuse-Violations-Possession,Gambling-Offenses
#Offenses-Against-Family-Children,Driving-Under-the-Influence,Liquor-Laws,Drunkenness
#Disorderly-Conduct,All-Other-Offenses-Except-Traffic,Suspicion,Curfew-Loitering-Law-Violations
#Run-a-ways,Vagrancy
def display_felony_html(county,year_tup):
    try:
        tmp_felony = juvenile_arrests.loc[idx[county,year_tup[0]:year_tup[1]], :][violent_felony]
        tmp_felony['Totals'] = tmp_felony.sum(axis=1)
        display(HTML('<center><h3>Violent Felony</h3></center>'))
        display(HTML(tmp_felony.transpose().to_html())) 
    except Exception as e:
        print('Error showing ORI data.'+str(e))

def display_lesser_html(county,year_tup):
    try:
        tmp_lesser = juvenile_arrests.loc[idx[county,year_tup[0]:year_tup[1]], :][lesser_offenses]
        tmp_lesser['Totals'] = tmp_lesser.sum(axis=1)
        display(HTML('<center><h3>Lesser Offenses</h3></center>'))
        display(HTML(tmp_lesser.transpose().to_html()))
    except Exception as e:
        print('Error showing ORI data.'+str(e))    
        
def display_school_html(county,year_tup):
    try:
        display(HTML(school.loc[idx[county,year_tup[0]:year_tup[1],year_tup[0]:year_tup[1]], :].transpose().to_html()))
    except Exception as e:
        print('Error showing School data.'+str(e))

def display_overview_html(county,year_tup):#NOT USED?
    try:
        display(HTML(overview.loc[idx[county,year_tup[0]:year_tup[1]], :].to_html()))
    except Exception as e:
        print('Error showing Overview data.'+str(e))

##PLACEMENTS
def display_placements_html(county,year_tup):
    #'period,Total Children in Care,Psychiatric RTC,Relative foster home,Supervised Independent Living,
    #Long Term FC Non Relative,Long Term FC Relative,Interim,Boys School,State Hospital,Girls School,
    #Hospital,Total Family-like setting,Therapeutic FC Non Relative,Specialized FC Relative,Jail,
    #Pre-adoptive home,Crisis Center,Detention,district,Therapeutic FC Relative,Trial Home Visit,
    #Runaway,Total Group Care,Non-relative foster home,Residential Treatment,Group Home,Specialized FC Non Relative,Unknown
    #     Supervised Independent Living <<---????
    #GROUP CATEGORIES(NON RELATIVE)
#     Psychiatric RTC
#     Long Term FC Non Relative
#     Interim,Boys School,State Hospital,Girls School
#     Hospital
#     Therapeutic FC Non Relative,Jail
#     Pre-adoptive home,Crisis Center,Detention,Trial Home Visit
#     Runaway,Total Group Care,Non-relative foster home,Residential Treatment
#     Group Home,Specialized FC Non Relative,Unknown
    #RELATIVE CATEGORIES
    #Long Term FC Relative,Total Family-like setting,Specialized FC Relative,Therapeutic FC Relative
    placement_fields_to_show=['Psychiatric RTC',
    'Long Term FC Non Relative',
    'Interim','Boys School','State Hospital','Girls School',
    'Hospital',
    'Therapeutic FC Non Relative','Jail',
    'Pre-adoptive home','Crisis Center','Detention','Trial Home Visit',
    'Runaway','Non-relative foster home','Residential Treatment',
    'Group Home','Specialized FC Non Relative','Total Group Care','Total Family-like setting','Unknown','Total Children in Care']
    try:
        district_num = 0
        for county_arr_idx in range(len(placement_county_districts)):#search through county list to match county name with judicial district number
            if county in placement_county_districts[county_arr_idx]:
                district_num = county_arr_idx
                #print('found county[{}:{}] in arr:{}'.format(county,district_num,str(placement_county_districts[district_num])))
     
        display(HTML(placement_data[placement_fields_to_show].loc[idx[year_tup[0]:year_tup[1],district_num+1], :].transpose().to_html()))
    except Exception as e:
        print('Error showing Placements data.'+str(e))

def display_county_placements_rates_html(county,year_tup):
    try:
        district_num = 0
        for county_arr_idx in range(1,len(placement_county_districts)):#search through county list to match county name with judicial district number
            if county in placement_county_districts[county_arr_idx]:
                district_num = county_arr_idx
                #print('found county[{}:{}] in arr:{}'.format(county,district_num,str(placement_county_districts[district_num])))
        #display(HTML(overview.loc[idx[county,year_tup[0]:year_tup[1]], :].to_html()))
        display(HTML(placement_rates_data.loc[idx[district_num+1,year_tup[0]:year_tup[1]], :].transpose().to_html()))
    except Exception as e:
        print('Error showing county placement rates data.'+str(e))

def display_state_placements_rates_html(county,year_tup):
    try:   
        display(HTML(state_placement_rates_data.loc[idx[year_tup[0]:year_tup[1]], :].transpose().to_html()))
    except Exception as e:
        print('Error showing state placement rate data.'+str(e))

##CASE COUNTS
def display_case_counts_html(county,year_tup):
    try:
        display(HTML(court_case_counts.loc[idx[str(year_tup[0]):str(year_tup[1]),county], :].transpose().to_html()))
    except Exception as e:
        print('Error showing Case Counts data.'+str(e))
        
def display_demographic_html(county,year_tup):
    try:
        display(HTML(demographic_data.loc[idx[county,:,year_tup[0]:year_tup[1]],:].transpose().to_html()))
    except Exception as e:
        print('Error showing demographic data.'+str(e))

In [671]:
# crime stats functions
#calculate crime rate per county
def county_crime_rate(county,year_tup,estimate_per):
    #df = juvenile_arrests.round({'Total Incidents':0,'Population 11-20':0,'Rate':2})
    display(HTML(juvenile_arrests[['Total Incidents','Population 11-17','Rate']].loc[idx[county,year_tup[0]:year_tup[1]], :].transpose().to_html())) 
    #display(HTML(df.loc[idx[county,year_tup[0]:year_tup[1]], :].transpose().to_html()))
    return

def state_crime_rate(county,year_tup,estimate_per):
    year_tup=(year_tup[0],year_tup[1])
    display(HTML(arrest_totals[['Total Incidents','Population 11-17','Rate']].loc[idx[year_tup[0]:year_tup[1]],:].transpose().to_html()))
    return

def agegroup_demographic(county,year_tup):
    display(HTML(agegroup_demographic_data[agegroup_demographic_data['Age Range']=='11-17'].loc[idx[county,year_tup[0]:year_tup[1]],:].transpose().to_html()))
    #display(HTML(agegroup_demographic_data[agegroup_demographic_data['Age Range']=='18-20'].loc[idx[county,year_tup[0]:year_tup[1]],:].transpose().to_html()))
    return

In [672]:
#plotly graphing functs
    # 5 year analysis
# display(HTML('<h2>5 Year State Analysis'))
# display(HTML('<p>A description of the producut goes here</p>'))
marker_colors = ['#6699ff','#00ff99','#669900','#ffcc00','#ff6600','#ff6600']
def generate_rate_data_obj(df,county,begin_year, end_year):
    years = [x for x in range(int(begin_year),int(end_year)+1)]

    #gather counties array
    counties = df.loc[idx[:,'2012':'2012'], :].index.values.tolist()
    counties = [x[0] for x in counties]
    #print(counties)

    #CREATE RATE GRAPH
    data = []
    
    cntr = 0
    #for year in years:
    #year_avg = np.mean(sum_total[cntr])
    data.append(go.Bar(
        #x = counties,
        x = years,
        y = df[['Rate']].loc[idx[county,begin_year:end_year], :].values.flatten(),
        #text=str(years),
        textposition = 'auto',
        #name=years,
        name='Number Incidents',
        marker=dict(
            color=marker_colors[cntr],
            line=dict(
                color='rgb(8,48,107)',
                width=1.5),
        ),
        opacity=0.6
        )
    )
#     data.append(go.Scatter(
#             x = years,
#             y = [year_avg for x in range(len(years))],
#             name='{} Average'.format(year),
#             marker=dict(
#             color=marker_colors[cntr],
#             line=dict(
#                 color='rgb(8,48,107)',
#                 width=1.5),
#         )
#         )
#     )
    
        #cntr+=1
    return data

def generate_pct_change_data_obj(df,county,begin_year, end_year, district=False):
    years = [x for x in range(int(begin_year),int(end_year)+1)]
#     if not district:
#         groups = counties
#     else:
#         groups = [x for x in range(1,10)]
    groups = years
    df['pct'] = df['Rate'].pct_change()
    
    objs = []
    data = []
    for year in years:
        pct_change_vals=df[['pct']].loc[idx[county,year:year], :].values.flatten()[0]
        objs.append({'year':year,'pct_change':pct_change_vals})

    cntr=0
    #for year in years:
    
    data.append(go.Bar(
        x = groups,
        y = [x['pct_change'] for x in objs],
        #text=,
        #textposition = 'auto',
        #name=['{} Percent Change'.format(year) for year in years],
        name='Number Incidents',
        marker=dict(
            color=marker_colors[cntr],
            line=dict(
                color='rgb(8,48,107)',
                width=1.5),
        ),
        opacity=0.6
        )
    )
    year_avg = np.mean([x['pct_change'] for x in objs])
    data.append(go.Scatter(
            x = groups,
            y = [year_avg for x in range(len(groups))],
            name='{} Average'.format(year),
            marker=dict(
            color=marker_colors[cntr],
            line=dict(
                color='rgb(8,48,107)',
                width=1.5),
        )
        )
    )

        #cntr+=1
    return data

# display(HTML('<h1>Statewide 5-year Comparison Charts</h1>'))
# display(HTML("""
# <p>
# The following graphs illustrate  a 5 year comparison placement, educational discipline, and juvenile arrest rates broken up by county and/or judicial district.
# </p>
# """))

In [673]:
def display_arrests_rate_bar(county,begin_year,end_year):
    display(HTML('<h2>Arrest Rates For {} County {}-{}</h2>'.format(county,begin_year,end_year)))
    #data = generate_rate_data_obj(juvenile_arrests,county,begin_year,end_year)
    years = [x for x in range(int(begin_year),int(end_year)+1)]

    #gather counties array
    counties = juvenile_arrests.loc[idx[:,'2012':'2012'], :].index.values.tolist()
    counties = [x[0] for x in counties]
    #print(counties)
    
    #CREATE RATE GRAPH
    #data = []
    b1 = go.Bar(
        #x = counties,
        x = years,
        y = juvenile_arrests[['Rate']].loc[idx[county,begin_year:end_year], :].values.flatten(),
        textposition = 'auto',
        name='Arrest Rate per 10,000',
        marker=dict(
            color=marker_colors[0],
            line=dict(
                color='rgb(8,48,107)',
                width=1.5),
        ),
        opacity=0.6
    )
#     state_rate_avg = []
#     for year in years:
#         arrest_totals[['Rate']].loc[idx[year:year],:].values.flatten().tolist()
        
    l1 = go.Scatter(
            x = years,
            y = arrest_totals[['Rate']].loc[idx[begin_year:end_year],:].values.flatten().tolist(),
            name='Average State Rate',
            marker=dict(
            color='#6699ff',
            line=dict(
                color='rgb(8,48,107)',
                width=1.5),
        )
    )
    #arrest_totals[['Rate']].loc[idx[begin_year:end_year],:]
    
    juvenile_arrests['pct'] = juvenile_arrests['Rate'].pct_change()
    
    objs = []
    #data = []
    for year in years:
        pct_change_vals=juvenile_arrests[['pct']].loc[idx[county,year:year], :].values.flatten()[0]
        objs.append({'year':year,'pct_change':pct_change_vals})

    cntr=0
    #for year in years:
    
    b2 = go.Bar(
        x = years,
        y = [x['pct_change'] for x in objs],
        xaxis='x2',
        yaxis='y2',
        name='% Change in Rate',
        marker=dict(
            color='#6699ff',
            line=dict(
                color='rgb(8,48,107)',
                width=1.5),
        ),
        opacity=0.6
    )
    
    year_average = []
    for year in years:
        year_average.append(np.mean(juvenile_arrests.loc[idx[:,year:year],:]['pct'].values.flatten().tolist())) 
        
    l2 = go.Scatter(
        x = years,
        y = year_average,
        xaxis='x2',
        yaxis='y2',
        name='Average State % Rate Change',
        marker=dict(
        color='#6699ff',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5)
        )
    )
        
    data = [b1,l1,b2,l2]
    layout = go.Layout(
        autosize=False,
        width=1000,
        height=600,
        xaxis = dict(
            domain=[0,0.45]
        ),
        xaxis2=dict(
            domain=[0.55,1]
        ),
        yaxis2=dict(
            anchor='x2'
        ),
    )
    pl.offline.iplot(go.Figure(data=data,layout=layout), filename='5year_county_arrest_rates')
    return

def display_arrests_percent_bar(county,begin_year,end_year):
#     display(HTML('<h2>Percent Change in Arrest Rates For {} County {}-{}</h2>'.format(county,begin_year,end_year)))
#     data = generate_pct_change_data_obj(juvenile_arrests,county,begin_year,end_year)
#     layout = go.Layout(
#         autosize=False,
#         width=500,
#         height=500
#     )
    
#     pl.offline.iplot(go.Figure(data=data,layout=layout), filename='5year_county_arrest_pct')
    return

In [674]:
def display_school_rate_bar(county,begin_year,end_year):
    display(HTML('<h2>School Discipline Rates For {} County {}-{}</h2>'.format(county,begin_year,end_year)))
    display(HTML("""
    <p>
    The graph below shows the number of educational discipline events(excluding in-school suspension) for each county
    for the time period between {}-{}. Each county is represented along the x-axis, with 5 vertical bars for each 
    county representing the educational discipline rate of that county for a particular year(vertical bars are 
    color coded in the legend on the right). This discipline rate is calculated as the number of incidents 
    per {} residents.The horizontal lines across the graph represent the average discipline rate for the
    entire state for a particular year(also color coded to correspond with year).
    </p>
    """.format(begin_year,end_year,ESTIMATES_PER)))
#     data = generate_rate_data_obj(school_county_rates,county,begin_year,end_year)
#     layout = go.Layout(
#         autosize=False,
#         width=500,
#         height=500
#     )
    
    years = [x for x in range(int(begin_year),int(end_year)+1)]

    #gather counties array
    counties = school_county_rates.loc[idx[:,'2012':'2012'], :].index.values.tolist()
    counties = [x[0] for x in counties]
    #print(counties)
    
    #CREATE RATE GRAPH
    #data = []
    b1 = go.Bar(
        #x = counties,
        x = years,
        y = school_county_rates[['Rate']].loc[idx[county,begin_year:end_year], :].values.flatten(),
        textposition = 'auto',
        name='Incident Rate per 10,000',
        marker=dict(
            color=marker_colors[0],
            line=dict(
                color='rgb(8,48,107)',
                width=1.5),
        ),
        opacity=0.6
    )
    rate_year_avg = []
    for year in years:
        rate_year_avg.append(np.mean(school_county_rates[['Rate']].loc[idx[:,year:year], :].values.flatten()))
        
    l1 = go.Scatter(
            x = years,
            y = rate_year_avg,
            name='Average State Rate',
            marker=dict(
            color='#6699ff',
            line=dict(
                color='rgb(8,48,107)',
                width=1.5),
        )
    )
    
    school_county_rates['pct'] = school_county_rates['Rate'].pct_change()
    
    objs = []
    #data = []
    pct_change_vals = []
#     for year in years:
#         pct_change_vals.append(school_county_rates[['pct']].loc[idx[county,year:year], :].values[0][0])
        #objs.append({'year':year,'pct_change':pct_change_vals})

    cntr=0
    #for year in years:
    
    b2 = go.Bar(
        x = years,
        y = school_county_rates[['pct']].loc[idx[county,begin_year:end_year], :].values.flatten().tolist(),
        xaxis='x2',
        yaxis='y2',
        name='% Change in Rate',
        marker=dict(
            color='#6699ff',
            line=dict(
                color='rgb(8,48,107)',
                width=1.5),
        ),
        opacity=0.6
    )
    
    year_average = []
    for year in years:
        #print(juvenile_arrests[['Rate']].loc[idx[:,year_tup[0]:year_tup[0]],:].values.flatten().tolist())
        year_average.append(np.mean(np.nan_to_num(school_county_rates[['pct']].loc[idx[:,year:year], :].values.flatten().tolist())))
        
    l2 = go.Scatter(
        x = years,
        y = year_average,
        xaxis='x2',
        yaxis='y2',
        name='Average State % Rate Change',
        marker=dict(
        color='#6699ff',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5)
        )
    )
        
    data = [b1,l1,b2,l2]
    layout = go.Layout(
        autosize=False,
        width=1000,
        height=600,
        xaxis = dict(
            domain=[0,0.45]
        ),
        xaxis2=dict(
            domain=[0.55,1]
        ),
        yaxis2=dict(
            anchor='x2'
        ),
    )
    
    pl.offline.iplot(go.Figure(data=data,layout=layout), filename='5year_county_school_rates')
    return

def display_school_percent_bar(county,begin_year,end_year):
#     display(HTML('<h2>Percent Change in School Discipline Rates For {} County {}-{}</h2>'.format(county,begin_year,end_year)))
#     data = generate_pct_change_data_obj(school_county_rates,county,begin_year,end_year)
#     layout = go.Layout(
#         autosize=False,
#         width=500,
#         height=500
#     )
    
#     pl.offline.iplot(go.Figure(data=data,layout=layout), filename='5year_county_school_pct')    
    return

In [675]:
def getDistrictFromCounty(county):
    district_num = 0
    for county_arr_idx in range(len(placement_county_districts)):#search through county list to match county name with judicial district number
        if county in placement_county_districts[county_arr_idx]:
            district_num = county_arr_idx
    return district_num+1

In [676]:
#Plotly Graph to HTML Display functs

#placement_rates_data.loc[idx[district_num+1,year_tup[0]:year_tup[1]], :]
#state_placement_rates_data.loc[idx[year_tup[0]:year_tup[1]], :]
def display_placement_rate_bar(county,begin_year,end_year):
    display(HTML('<h2>Placement Rates For {} County {}-{}</h2>'.format(county,begin_year,end_year)))
    #data = generate_rate_data_obj(school_county_rates,county,begin_year,end_year)
    years = [x for x in range(int(begin_year),int(end_year)+1)]
    
    district_num = getDistrictFromCounty(county)
    
    #CREATE RATE GRAPH
    b1 = go.Bar(
        #x = counties,
        x = years,
        y = placement_rates_data['Rate'].loc[idx[district_num,begin_year:end_year]].values.flatten().tolist(),
        textposition = 'auto',
        name='Placement Rate per 10,000',
        marker=dict(
            color=marker_colors[0],
            line=dict(
                color='rgb(8,48,107)',
                width=1.5),
        ),
        opacity=0.6
    )
    rate_year_avg = []
    for year in years:
        rate_year_avg.append(np.mean(state_placement_rates_data['Rate'].loc[idx[year:year]].values.flatten()))
        
    l1 = go.Scatter(
            x = years,
            y = rate_year_avg,
            name='Average State Rate',
            marker=dict(
            color='#6699ff',
            line=dict(
                color='rgb(8,48,107)',
                width=1.5),
        )
    )
    
    placement_rates_data['pct'] = placement_rates_data['Rate'].pct_change()
    state_placement_rates_data['pct'] = state_placement_rates_data['Rate'].pct_change()
                                     
    objs = []
    #data = []
    pct_change_vals = []
#     for year in years:
#         pct_change_vals.append(school_county_rates[['pct']].loc[idx[county,year:year], :].values[0][0])
        #objs.append({'year':year,'pct_change':pct_change_vals})

    cntr=0
    #for year in years:
    
    b2 = go.Bar(
        x = years,
        y = placement_rates_data['pct'].loc[idx[district_num,begin_year:end_year]].values.flatten().tolist(),
        xaxis='x2',
        yaxis='y2',
        name='% Change in Rate',
        marker=dict(
            color='#6699ff',
            line=dict(
                color='rgb(8,48,107)',
                width=1.5),
        ),
        opacity=0.6
    )
    
    year_average = []
    for year in years:
        #print(juvenile_arrests[['Rate']].loc[idx[:,year_tup[0]:year_tup[0]],:].values.flatten().tolist())
        year_average.append(np.mean(np.nan_to_num(state_placement_rates_data['pct'].loc[idx[year:year]].values.flatten().tolist())))
        
    l2 = go.Scatter(
        x = years,
        y = year_average,
        xaxis='x2',
        yaxis='y2',
        name='Average State % Rate Change',
        marker=dict(
        color='#6699ff',
        line=dict(
            color='rgb(8,48,107)',
            width=1.5)
        )
    )
        
    data = [b1,l1,b2,l2]
    layout = go.Layout(
        autosize=False,
        width=1000,
        height=600,
        xaxis = dict(
            domain=[0,0.45]
        ),
        xaxis2=dict(
            domain=[0.55,1]
        ),
        yaxis2=dict(
            anchor='x2'
        ),
    )
    
    pl.offline.iplot(go.Figure(data=data,layout=layout), filename='5year_county_placement_rates')
    return

def display_placement_percent_bar(county,begin_year,end_year):
#     display(HTML('<h2>Percent Change in Placement Rates For {} County {}-{}</h2>'.format(county,begin_year,end_year)))
#     data = generate_pct_change_data_obj(school_county_rates,county,begin_year,end_year)
#     pl.offline.iplot(data, filename='5year_county_placement_pct') 
    return


In [677]:

##Widget Objects
COUNTY = 'Albany'
county_drop = widgets.Dropdown(
    options=[x for x in school.index.levels[0].str.strip()],
    description='County:',
    disabled=False,
)
YEAR_TUP = (2014,2017)
year_slide = widgets.IntRangeSlider(
    description='Year range:',
    min=2012,
    max=2017,
    value=YEAR_TUP,
    step=1
)
ESTIMATES_PER = 10000
estimates_slide = widgets.Select(
    options=[10000],
    value=ESTIMATES_PER,
    description='Estimate per',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True
)

In [678]:

#tables widgets
afcars_out = widgets.interactive_output(display_afcars_html,{'county':county_drop,'year_tup':year_slide})
county_out = widgets.interactive_output(display_county_html,{'county':county_drop,'year_tup':year_slide})
felony_out = widgets.interactive_output(display_felony_html,{'county':county_drop,'year_tup':year_slide})
lesser_out = widgets.interactive_output(display_lesser_html,{'county':county_drop,'year_tup':year_slide})
school_out = widgets.interactive_output(display_school_html,{'county':county_drop,'year_tup':year_slide})
placements_out = widgets.interactive_output(display_placements_html,{'county':county_drop,'year_tup':year_slide})
placements_rates_out = widgets.interactive_output(display_county_placements_rates_html,{'county':county_drop,'year_tup':year_slide})
state_placements_rates_out = widgets.interactive_output(display_state_placements_rates_html,{'county':county_drop,'year_tup':year_slide})
demographics_out = widgets.interactive_output(display_demographic_html,{'county':county_drop,'year_tup':year_slide})
case_counts_out = widgets.interactive_output(display_case_counts_html,{'county':county_drop,'year_tup':year_slide})
agegroup_demographic_out = widgets.interactive_output(agegroup_demographic,{'county':county_drop,'year_tup':year_slide})
#estimates widgets
crime_rate_out = widgets.interactive_output(county_crime_rate,{'county':county_drop,'year_tup':year_slide,'estimate_per':estimates_slide})
state_crime_rate_out = widgets.interactive_output(state_crime_rate,{'county':county_drop,'year_tup':year_slide,'estimate_per':estimates_slide})

#show everything
##################################### Header stuff
display(HTML('<h1>{} County Juvenile Justice 5 Year County Analysis Report for {}-{}</h1>'.format(COUNTY,YEAR_TUP[0],YEAR_TUP[1])))

##################################### Arrests Data
display(
    widgets.VBox([
        widgets.HTML(value='<hr><center><h1>ORI Arrests Data</h1></center>'),
        widgets.HTML(value="Wyoming Juvenile Arrests reported from agency ORI numbers. Split between violent felony offenses and non-violent offenses.")
    ]),
        widgets.HBox([
        felony_out,
        lesser_out
    ]),
    #widgets.HTML(value="SOURCE: Wyoming DCI UCR arrests by ORI"),
    widgets.HTML(value="<p class=\"page-break\"></p>")
)

display(
    widgets.HTML(value="<center><h1>Arrest Rate Comparisons</h1></center>"),
    widgets.HTML(value="Depicts county arrest rates calculated from the above ORI Arrests Data. Rate is calculated from \
        the 'Estimate per "+str(estimates_slide.value)+" people' field at the top of the form. County arrest rates are calculated \
        using the sum of all arrests for each year for the selected county portrayed in the ORI Arrests Data above. Then for \
        comparison, the state arrest rates are calculated from all counties for the selected years. Population data is \
        then added from the OJJDP source containing only population values for ages 11-17 within the county."),
    widgets.HBox([
        widgets.VBox([
            widgets.HTML(value="<h2>County Arrest Rates</h2>"),
            widgets.HBox([crime_rate_out]),
        ]),
        widgets.VBox([
            widgets.HTML(value="<h2>State Arrest Rates</h2><br>"),
            widgets.HBox([state_crime_rate_out]),
        ])
    ]),
    widgets.HBox([
        widgets.VBox([
                widgets.HTML(value="<h2>Juvenile Court Case Counts</h2>"),
                widgets.HTML(value="* denotes case counts greater than zero and less than five."),
                widgets.HBox([case_counts_out]),
                #widgets.HTML(value="SOURCE: OOJDP Easy Access to State and County Juvenile Court Case Counts")
            ]),
        widgets.VBox([
            #widgets.HTML(value="SOURCE: Wyoming DCI UCR arrests by ORI"),
            widgets.HTML(value="<h2>Juvenile Population Demographics by Age Group</h2>"),
            widgets.HBox([agegroup_demographic_out])
        ])
    ]),
    widgets.HTML(value="<p class=\"page-break\"></p>")
)
display(HTML('<p class="page-break"></p>'))
##############ARRESTS GRAPHS 
display_arrests_rate_bar(COUNTY,YEAR_TUP[0],YEAR_TUP[1])
display_arrests_percent_bar(COUNTY,YEAR_TUP[0],YEAR_TUP[1])
display(HTML('<p class="page-break"></p>'))

##################################### Placement Info
display(
    widgets.HTML(value="<hr><h1>DFS Placements</h1>"),
    widgets.HTML(value="The left table depicts placement type (of those in care) for probation/delinquency cases only,\
                 by Judicial District. The yearly numbers depicted are calculated from the 12-month average of the \
                 number of juveniles in a placement scenario, as reported at the end of every month. \
                 Data shown is for the judicial district to which the current selected \
                 county belongs to. The table on the right shows DFS placements by county as provided by \
                 the DFS prior to 2017(see note below). Placement rates are calculated using the same method \
                 as the arrest rates above."),
    widgets.HBox([    
        widgets.VBox([
            placements_out,
            #widgets.HTML(value="SOURCE: placements footer"),
        ]), 
        widgets.VBox([
            widgets.HTML(value="<h2>District Placement Rates</h2>"),
            placements_rates_out,
            widgets.HTML(value="<h2>State Placement Rates</h2>"),
            state_placements_rates_out,
            widgets.VBox([
                county_out,
                #widgets.HTML(value="SOURCE: by county footer"),
                widgets.HTML(value="Prior to 2017 all DFS reports were reported on a per county basis. After 2017, DFS data collection systems no longer classified data per county.")
            ])
    #         widgets.HBox([crime_rate_out])
        ]),
    ]),
    widgets.HTML(value="<p class=\"page-break\"></p>")
)
##############PLACEMENTS GRAPHS 
display_placement_rate_bar(COUNTY,YEAR_TUP[0],YEAR_TUP[1])
#display_arrests_percent_bar(COUNTY,YEAR_TUP[0],YEAR_TUP[1])
display(HTML('<p class="page-break"></p>'))
##################################### School info
display(
    widgets.VBox([
        widgets.HTML(value='<hr><h2>Educational Discipline Data</h2>')
    ]),
    widgets.VBox([
        school_out,
        #widgets.HTML(value="SOURCE: edu footnote")
    ])
)
display(HTML('<p class="page-break"></p>'))
display_school_rate_bar(COUNTY,YEAR_TUP[0],YEAR_TUP[1])
display_school_percent_bar(COUNTY,YEAR_TUP[0],YEAR_TUP[1])


HTML(value='<p class="page-break"></p>')

HTML(value='<center><h1>Arrest Rate Comparisons</h1></center>')

HTML(value="Depicts county arrest rates calculated from the above ORI Arrests Data. Rate is calculated from   …

HTML(value='<p class="page-break"></p>')

HTML(value='<hr><h1>DFS Placements</h1>')

HTML(value='The left table depicts placement type (of those in care) for probation/delinquency cases only,    …

HTML(value='<p class="page-break"></p>')

In [679]:
## Download Data
# dfunct = DataFunctions()
# overview = dfunct.getOverview()
# dfs = dfunct.getDFS()#afcars, county, plc
# school = dfunct.getSchool()
# agegroup_demographic_data = dfunct.getDemographic_By_AgeGroup()
# juvenile_arrests = dfunct.getORIData(agegroup_demographic_data)
# population_data = dfunct.getPopulationData()#phasing out?
# arrest_totals = dfunct.getStateTotalArrests(juvenile_arrests,population_data)
# judicial_district_population_data = dfunct.getJudicialDistrictJuvenilePopulations()
# placement_data = dfunct.getJudicialPlacementData()
# placement_rates_data,state_placement_rates_data = dfunct.getJudicialPlacementRates(placement_data,judicial_district_population_data)
# demographic_data = dfunct.getDemographicData()
# court_case_counts = dfunct.getCourtCaseNumbersData()
from IPython.display import Javascript

def downloadCSV_python(name):
    print('pycsv:{}'.format(name))
    csv_string = ''
    if name is 'Education':
        csv_string = school.to_csv(index=False).replace('\n','\\n').replace("'","\'")
    elif name is 'Demographic':
        csv_string = agegroup_demographic_data.to_csv(index=False).replace('\n','\\n').replace("'","\'")
    elif name is 'JuvyArrest':
        csv_string = arrest_totals.to_csv(index=False).replace('\n','\\n').replace("'","\'")
    elif name is 'JuvyCourtCase':
        csv_string = court_case_counts.to_csv(index=False).replace('\n','\\n').replace("'","\'")
        
    js_download = """
    var csv = '%s';

    var filename = 'results.csv';
    var blob = new Blob([csv], { type: 'text/csv;charset=utf-8;' });
    if (navigator.msSaveBlob) { // IE 10+
        navigator.msSaveBlob(blob, filename);
    } else {
        var link = document.createElement("a");
        if (link.download !== undefined) { // feature detection
            // Browsers that support HTML5 download attribute
            var url = URL.createObjectURL(blob);
            link.setAttribute("href", url);
            link.setAttribute("download", filename);
            link.style.visibility = 'hidden';
            link.innerHTML = 'link';
            document.body.appendChild(link);
            link.click();
            document.body.removeChild(link);
        }
    }
    """ % csv_string

    Javascript(js_download)
    
js_download_funct = """
<script>
function downloadCSV_js(name){
    console.log('clicked')
    var kernel = IPython.notebook.kernel;
    kernel.execute(downloadCSV_python(name));
}
</script>
"""
# display(HTML(js_download_funct))
# display(HTML('<h3>Download the Data</h3> Click on the links below to download the CSV files being used in the above report.'))
# display(HTML('<a onclick="downloadCSV_js(\'Education\')">Download Educational Discipline Data</a>'))
# display(HTML('<a onclick="downloadCSV_js(\'Demographic\')">Download Population and Demographic Information by Age Group Data</a>'))
# display(HTML('<a onclick="downloadCSV_js(\'JuvyArrest\')">Download Juvenile Arrest Data Data</a>'))
# display(HTML('<a onclick="downloadCSV_js(\'JuvyCourtCase\')">Download Juvenile Court Case Counts Data</a>'))

In [680]:
display(HTML('<p>Please visit <a href="https://github.com/coolcomputers/WyJJD/new-data/DataGuide.md" target="_blank">the Data Guide page</a> for data source citations.</p>'))
display(HTML('<p>Project created by Cool Computers for the Wyoming Children\'s Law Center, funded partially by grants provided by the <a href="https://www.aecf.org/where-we-work/location/wy/" target="_blank">Annie E. Casey Foundation</a></p>'))

In [681]:
#arrest_totals[['Total Incidents','Population 11-17','Rate']]
